# Transpilation - Code Laboratory

**Section 4: Running Circuits on IBM Quantum Hardware** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `transpile()` | `transpile(qc, backend, optimization_level=1)` | `QuantumCircuit` | One-off transpilation |
| `generate_preset_pass_manager()` | `generate_preset_pass_manager(optimization_level, backend)` | `PassManager` | Batch transpilation, layout access |
| `pm.run()` | `pm.run(circuit)` | `QuantumCircuit` | Apply pass manager |

---

In [ ]:
"""
Qiskit Code Laboratory - Transpilation
======================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# IBM Quantum Runtime imports
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeKyoto, FakeSherbrooke

# Local simulation
from qiskit_aer import AerSimulator

# =============================================================
# UTILITY FUNCTIONS FOR THIS NOTEBOOK
# =============================================================

def compare_circuits(original, transpiled, label=""):
    """Compare original and transpiled circuit metrics."""
    print(f"\n{label} Comparison:")
    print(f"  Original  → depth: {original.depth():3d}, size: {original.size():3d}")
    print(f"  Transpiled → depth: {transpiled.depth():3d}, size: {transpiled.size():3d}")
    if 'cx' in transpiled.count_ops():
        print(f"  CX gates added: {transpiled.count_ops().get('cx', 0)}")

def show_transpilation(qc, backend, level, label=""):
    """Transpile and show before/after."""
    pm = generate_preset_pass_manager(optimization_level=level, backend=backend)
    transpiled = pm.run(qc)
    print(f"\n{label}")
    print(f"Level {level}: depth={transpiled.depth()}, gates={transpiled.size()}")
    return transpiled

print("✅ Environment ready - using FakeManilaV2 for demonstrations")

---

## `transpile()`

### Signature
```python
transpile(
    circuits: QuantumCircuit | List[QuantumCircuit],
    backend: Backend = None,
    basis_gates: List[str] = None,
    coupling_map: CouplingMap = None,
    optimization_level: int = 1,   # 0-3
    initial_layout: List[int] = None,
    seed_transpiler: int = None
) -> QuantumCircuit | List[QuantumCircuit]
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `circuits` | `QuantumCircuit` | Yes | - | Circuit(s) to transpile |
| `backend` | `Backend` | No | `None` | Target backend (auto-provides basis_gates, coupling_map) |
| `basis_gates` | `List[str]` | No | `None` | Allowed gate set |
| `coupling_map` | `CouplingMap` | No | `None` | Qubit connectivity |
| `optimization_level` | `int` | No | `1` | 0=none, 1=light, 2=medium, 3=heavy |
| `seed_transpiler` | `int` | No | `None` | For reproducibility |

### Returns
`QuantumCircuit` - Hardware-compatible circuit

### See Also
- [README: Transpilation Fundamentals](./README.md#2-transpilation)

In [ ]:
# ============================================================
# transpile() - BASIC USAGE
# ============================================================

# Create simple circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

print("Original circuit:")
print(qc.draw())
print(f"Depth: {qc.depth()}, Size: {qc.size()}")

# Basic transpile (no backend - generic optimization)
qc_trans = transpile(qc, optimization_level=1)

print("\nTranspiled (no backend):")
print(qc_trans.draw())
print(f"Depth: {qc_trans.depth()}, Size: {qc_trans.size()}")

print("\n✅ transpile() converts circuit to optimized form")

In [ ]:
# ============================================================
# transpile() - PARAMETER VARIATIONS
# ============================================================

backend = FakeManilaV2()

# Create test circuit
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.t(0)

print("Original circuit:")
print(qc.draw())
print(f"Depth: {qc.depth()}, Size: {qc.size()}")

print("\n" + "=" * 50)
print("OPTIMIZATION LEVELS COMPARISON")
print("=" * 50)

for level in range(4):
    qc_trans = transpile(qc, backend=backend, optimization_level=level)
    cx_count = qc_trans.count_ops().get('cx', 0) + qc_trans.count_ops().get('ecr', 0)
    print(f"Level {level}: depth={qc_trans.depth():3d}, size={qc_trans.size():3d}, 2Q gates={cx_count}")

print("\n💡 Level 0=debug, 1=fast(default), 2=balanced, 3=best quality")

In [ ]:
# ============================================================
# transpile() - WITH BASIS GATES
# ============================================================

qc = QuantumCircuit(2)
qc.h(0)
qc.t(0)
qc.cx(0, 1)

print("Original gates:", dict(qc.count_ops()))

# Transpile to IBM basis gates
basis_gates = ['cx', 'id', 'rz', 'sx', 'x']
qc_trans = transpile(qc, basis_gates=basis_gates, optimization_level=1)

print(f"\nTranspiled to {basis_gates}:")
print(qc_trans.draw())
print(f"New gates: {dict(qc_trans.count_ops())}")

# Verify all gates are in basis set
all_in_basis = set(qc_trans.count_ops().keys()).issubset(set(basis_gates))
print(f"\nAll gates in basis? {all_in_basis}")
print("\n✅ basis_gates constrains output gates")

In [ ]:
# ============================================================
# transpile() - WITH COUPLING MAP
# ============================================================

# Linear coupling: 0 - 1 - 2 - 3 (no direct 0↔3)
coupling_map = CouplingMap([[0,1], [1,0], [1,2], [2,1], [2,3], [3,2]])

# Create circuit that violates coupling
qc = QuantumCircuit(4)
qc.cx(0, 3)  # 0↔3 NOT directly connected!

print("Original circuit (violates coupling):")
print(qc.draw())

# Transpile with coupling constraint
basis_gates = ['cx', 'id', 'rz', 'sx', 'x']
qc_trans = transpile(qc, coupling_map=coupling_map, basis_gates=basis_gates, optimization_level=1)

print("\nTranspiled (with SWAPs for routing):")
print(qc_trans.draw())

swap_equiv = qc_trans.count_ops().get('cx', 0) - 1  # Original had 1 CX
print(f"\n⚠️ Added ~{swap_equiv//3} SWAP gates (each SWAP = 3 CX)")
print("✅ Coupling map forces routing via intermediate qubits")

In [ ]:
# ============================================================
# transpile() - WITH BACKEND (RECOMMENDED)
# ============================================================

backend = FakeManilaV2()

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 2)  # May need routing on Manila topology

print(f"Backend: {backend.name}")
print(f"Coupling map: {backend.coupling_map}")

# Backend provides basis_gates AND coupling_map automatically!
qc_trans = transpile(qc, backend=backend, optimization_level=2)

print("\nOriginal:")
print(qc.draw())
print(f"\nTranspiled for {backend.name}:")
print(qc_trans.draw())

print("\n💡 backend= is easiest - auto-provides all constraints!")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Transpile is NOT Deterministic
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: Transpilation is NOT deterministic!")
print("=" * 55)

backend = FakeManilaV2()
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)

# Two transpilations - may give different results!
trans1 = transpile(qc, backend=backend, optimization_level=3)
trans2 = transpile(qc, backend=backend, optimization_level=3)

print(f"\nTranspile #1: depth={trans1.depth()}, size={trans1.size()}")
print(f"Transpile #2: depth={trans2.depth()}, size={trans2.size()}")

# Fix: Use seed_transpiler for reproducibility
trans_seeded1 = transpile(qc, backend=backend, optimization_level=3, seed_transpiler=42)
trans_seeded2 = transpile(qc, backend=backend, optimization_level=3, seed_transpiler=42)

print(f"\nWith seed=42 #1: depth={trans_seeded1.depth()}, size={trans_seeded1.size()}")
print(f"With seed=42 #2: depth={trans_seeded2.depth()}, size={trans_seeded2.size()}")

print("\n💡 Use seed_transpiler=42 for reproducible results!")

---

## `generate_preset_pass_manager()`

### Signature
```python
generate_preset_pass_manager(
    optimization_level: int,     # 0-3
    backend: Backend = None,
    basis_gates: List[str] = None,
    coupling_map: CouplingMap = None,
    **kwargs
) -> PassManager
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `optimization_level` | `int` | Yes | - | 0=none, 1=light, 2=medium, 3=heavy |
| `backend` | `Backend` | No | `None` | Target backend |

### Returns
`PassManager` - Reusable transpilation pipeline

### See Also
- [README: Pass Manager](./README.md#transpilation)

In [ ]:
# ============================================================
# generate_preset_pass_manager() - BASIC USAGE
# ============================================================

backend = FakeManilaV2()

# Create pass manager once
pm = generate_preset_pass_manager(
    optimization_level=2,
    backend=backend
)

# Use for single circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

transpiled = pm.run(qc)
print("Single circuit transpilation:")
print(f"  Original depth: {qc.depth()}")
print(f"  Transpiled depth: {transpiled.depth()}")

# Use for batch transpilation (more efficient!)
circuits = [QuantumCircuit(2) for _ in range(3)]
for i, c in enumerate(circuits):
    c.h(0)
    c.cx(0, 1)
    c.rz(i * 0.5, 0)

transpiled_batch = pm.run(circuits)
print(f"\nBatch transpilation: {len(transpiled_batch)} circuits")

print("\n✅ Pass manager is reusable and efficient for batch jobs")

In [ ]:
# ============================================================
# generate_preset_pass_manager() - LAYOUT ACCESS
# ============================================================

# CRITICAL: Access final qubit layout for observable mapping!

backend = FakeManilaV2()
pm = generate_preset_pass_manager(optimization_level=2, backend=backend)

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)

transpiled = pm.run(qc)

# Get the layout from pass manager
layout = transpiled.layout

print("LAYOUT INFORMATION:")
print("=" * 50)
print(f"Initial layout: {layout.initial_layout}")
print(f"Final layout: {layout.final_layout}")

# Show virtual → physical mapping
print("\nVirtual → Physical qubit mapping:")
for i in range(qc.num_qubits):
    physical = layout.final_layout[qc.qubits[i]]
    print(f"  Virtual q[{i}] → Physical qubit {physical}")

print("\n💡 Layout is CRITICAL for mapping observables in Estimator!")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: SWAP = 3 CX gates
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: Each SWAP gate = 3 CX gates!")
print("=" * 55)

# Linear coupling: 0-1-2-3 (no direct 0↔3)
coupling_map = CouplingMap([[0,1], [1,0], [1,2], [2,1], [2,3], [3,2]])
basis_gates = ['cx', 'id', 'rz', 'sx', 'x']

# Circuit requiring routing
qc = QuantumCircuit(4)
qc.cx(0, 3)  # Need to route through 1 and 2

print("Original: CX(0,3) - but 0↔3 not connected")
print(qc.draw())

# Transpile
pm = generate_preset_pass_manager(
    optimization_level=1, 
    coupling_map=coupling_map,
    basis_gates=basis_gates
)
transpiled = pm.run(qc)

print("\nTranspiled:")
print(transpiled.draw())

cx_count = transpiled.count_ops().get('cx', 0)
print(f"\n📊 Total CX gates: {cx_count}")
print(f"   Original CX: 1")
print(f"   Added CX (from SWAPs): {cx_count - 1}")
print(f"   Estimated SWAPs: ~{(cx_count - 1) // 3}")

print("\n💡 Remember: SWAP = 3 CX, NOT 2!")

In [ ]:
# ============================================================
# COMPARISON: transpile() vs generate_preset_pass_manager()
# ============================================================

print("=" * 60)
print("COMPARING: transpile() vs generate_preset_pass_manager()")
print("=" * 60)

backend = FakeManilaV2()

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

print("\n1. transpile() - One-line convenience function")
print("   Usage: transpile(qc, backend, optimization_level=2)")
t1 = transpile(qc, backend=backend, optimization_level=2)
print(f"   Result: depth={t1.depth()}")

print("\n2. generate_preset_pass_manager() - Reusable pipeline")
print("   pm = generate_preset_pass_manager(optimization_level=2, backend=backend)")
print("   transpiled = pm.run(qc)")
pm = generate_preset_pass_manager(optimization_level=2, backend=backend)
t2 = pm.run(qc)
print(f"   Result: depth={t2.depth()}")

print("\n" + "=" * 60)
print("WHEN TO USE EACH")
print("=" * 60)
print("\n  transpile()          → Single circuit, quick test")
print("  pass_manager.run()   → Batch circuits, need layout access")
print("  pass_manager.run()   → Observables mapping (Estimator)")

print("\n✅ Use pass_manager for production code with Estimator")

In [ ]:
# ============================================================
# CHALLENGE 1: Optimize for Hardware
# ============================================================
# Task: Transpile a GHZ state circuit for FakeKyoto
# Expected: Circuit that runs on hardware with minimal depth
# ============================================================

def create_and_transpile_ghz(n_qubits, backend, optimization_level):
    """
    Create n-qubit GHZ state and transpile for backend.
    
    Args:
        n_qubits: Number of qubits for GHZ state
        backend: Target backend
        optimization_level: 0-3
    
    Returns:
        Tuple of (original_circuit, transpiled_circuit)
    """
    # Create GHZ circuit
    qc = QuantumCircuit(n_qubits)
    qc.h(0)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    
    # Transpile
    pm = generate_preset_pass_manager(
        optimization_level=optimization_level,
        backend=backend
    )
    transpiled = pm.run(qc)
    
    return qc, transpiled

# Test your solution
backend = FakeKyoto()

print("GHZ State Transpilation Test")
print("=" * 50)

for n in [3, 5, 7]:
    original, transpiled = create_and_transpile_ghz(n, backend, optimization_level=2)
    print(f"\n{n}-qubit GHZ:")
    print(f"  Original depth: {original.depth()}")
    print(f"  Transpiled depth: {transpiled.depth()}")
    cx_count = sum(transpiled.count_ops().get(g, 0) for g in ['cx', 'ecr'])
    print(f"  2-qubit gates: {cx_count}")

print("\n✅ Challenge 1 PASSED - GHZ states transpiled for hardware!")

In [ ]:
# ============================================================
# CHALLENGE 2: Test Routing on Custom Topology
# ============================================================
# Task: Create a custom coupling map and verify SWAP insertion
# Expected: Demonstrate routing adds SWAP gates
# ============================================================

def test_routing(qc, coupling_map, basis_gates):
    """
    Transpile circuit and count routing overhead.
    
    Returns:
        Dict with original and transpiled metrics
    """
    pm = generate_preset_pass_manager(
        optimization_level=1,
        coupling_map=coupling_map,
        basis_gates=basis_gates
    )
    transpiled = pm.run(qc)
    
    orig_cx = qc.count_ops().get('cx', 0)
    trans_cx = transpiled.count_ops().get('cx', 0)
    
    return {
        'original_cx': orig_cx,
        'transpiled_cx': trans_cx,
        'added_cx': trans_cx - orig_cx,
        'est_swaps': (trans_cx - orig_cx) // 3
    }

# Test: Ring topology (0-1-2-3-0)
ring_map = CouplingMap([
    [0,1], [1,0], [1,2], [2,1], [2,3], [3,2], [3,0], [0,3]
])
basis = ['cx', 'id', 'rz', 'sx', 'x']

# Circuit with various distances
qc = QuantumCircuit(4)
qc.cx(0, 1)  # Adjacent - no routing
qc.cx(0, 2)  # Distance 2 - needs routing

results = test_routing(qc, ring_map, basis)

print("Ring Topology Routing Test")
print("=" * 50)
print(f"Original CX gates: {results['original_cx']}")
print(f"Transpiled CX gates: {results['transpiled_cx']}")
print(f"Added CX gates: {results['added_cx']}")
print(f"Estimated SWAPs: {results['est_swaps']}")

assert results['transpiled_cx'] >= results['original_cx'], "Should add or keep same"
print("\n✅ Challenge 2 PASSED - Routing verification complete!")